In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install -U langchain-community
# !pip install -U transformers
# !pip install -U bitsandbytes accelerate
# # !pip install datasets
# !pip install pypdf
# !pip install peft
# !pip install langchain
# !pip install langchain-huggingface faiss-cpu
# !pip install rank_bm25
# !pip install langchain-huggingface

In [ ]:
# !pip install --upgrade --force-reinstall autoawq --extra-index-url https://download.pytorch.org/whl/cu121
# !pip uninstall -y autoawq
# !git clone https://github.com/casper-hansen/AutoAWQ.git
# %cd AutoAWQ
# !pip install .

In [ ]:
import re
import json
import torch
import pandas as pd
import numpy as np
import glob

import tqdm
from statistics import mean
from datasets import Dataset, load_dataset
from typing import List, Dict

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft import PeftModelForCausalLM
from peft import AutoPeftModelForCausalLM
from peft import prepare_model_for_kbit_training

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline, BitsAndBytesConfig
from transformers import EarlyStoppingCallback
from transformers import default_data_collator

from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from rank_bm25 import BM25Okapi
import pickle

from google.colab import files
import warnings
warnings.filterwarnings("ignore")

# retriever

## pdf load & split chunks

In [ ]:
# pdf 로드
pdf_folder = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/pdf/*.pdf"
pdf_files = glob.glob(pdf_folder)

In [ ]:
all_chunks = []

def split_chunks(pdf_files):
  for pdf_file in tqdm.tqdm(pdf_files, desc = 'Pdf Load'):
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()

    # 불필요한 문자 제거
    pattern = r'[①-⑳◆·「」｢｣□◦\-\*❶-❸Ⅲ※‧Ÿ]'
    for doc in docs:
      text = doc.page_content
      text = re.sub(pattern, ' ', text)
      text = re.sub(r'\n', ' ', text)
      text = re.sub(r'\s+', ' ', text)
      text = text.strip()
      doc.page_content = text

    paragraph_lengths = [len(doc.page_content) for doc in docs]

    print()
    print("문단 개수:", len(paragraph_lengths))
    print("평균 문단 길이:", round(mean(paragraph_lengths)))
    print("최소 문단 길이:", min(paragraph_lengths))
    print("최대 문단 길이:", max(paragraph_lengths))

    # 추천 chunk_size & overlap 계산
    avg_len = mean(paragraph_lengths)
    max_len = max(paragraph_lengths)

    # chunk_size는 최대 문단 길이보다 조금 크게, 평균의 2~3배 정도로 설정
    chunk_size = int(min(max_len * 1.1, avg_len * 3))
    chunk_overlap = int(chunk_size * 0.15)  # 15% 정도 겹치기

    print("추천 chunk_size:", chunk_size)
    print("추천 chunk_overlap:", chunk_overlap)

    text_spliter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap,
      separators = ["\n\n", "\n", " ", ""]
      )

    split_chunks = text_spliter.split_documents(docs)
    print("청크의 수 :", len(split_chunks))
    all_chunks.extend(split_chunks)

In [ ]:
split_chunks(pdf_files)

Pdf Load:   8%|▊         | 1/12 [00:13<02:31, 13.80s/it]


문단 개수: 28
평균 문단 길이: 1494
최소 문단 길이: 55
최대 문단 길이: 3228
추천 chunk_size: 3550
추천 chunk_overlap: 532
청크의 수 : 28


Pdf Load:  25%|██▌       | 3/12 [00:18<00:42,  4.70s/it]


문단 개수: 112
평균 문단 길이: 514
최소 문단 길이: 0
최대 문단 길이: 1167
추천 chunk_size: 1283
추천 chunk_overlap: 192
청크의 수 : 99

문단 개수: 17
평균 문단 길이: 1629
최소 문단 길이: 1019
최대 문단 길이: 1996
추천 chunk_size: 2195
추천 chunk_overlap: 329
청크의 수 : 17

문단 개수: 6
평균 문단 길이: 1439
최소 문단 길이: 178
최대 문단 길이: 1780
추천 chunk_size: 1958
추천 chunk_overlap: 293
청크의 수 : 6


Pdf Load:  42%|████▏     | 5/12 [00:19<00:14,  2.13s/it]

...


문단 개수: 732
평균 문단 길이: 986
최소 문단 길이: 0
최대 문단 길이: 2910
추천 chunk_size: 2956
추천 chunk_overlap: 443
청크의 수 : 720


Pdf Load:  92%|█████████▏| 11/12 [01:07<00:11, 11.55s/it]


문단 개수: 166
평균 문단 길이: 837
최소 문단 길이: 0
최대 문단 길이: 1588
추천 chunk_size: 1746
추천 chunk_overlap: 261
청크의 수 : 163


Pdf Load: 100%|██████████| 12/12 [01:09<00:00,  5.83s/it]


문단 개수: 63
평균 문단 길이: 433
최소 문단 길이: 0
최대 문단 길이: 1092
추천 chunk_size: 1201
추천 chunk_overlap: 180
청크의 수 : 61


## build retriever

In [ ]:
def build_retriever(all_chunks,
                               embedding_model: str = "sentence-transformers/all-mpnet-base-v2"
                               ):
  # FAISS retriever
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                     cache_folder="/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/faiss_embedding_models")
  faiss_db = FAISS.from_documents(all_chunks, embeddings)
  faiss_retriever = faiss_db.as_retriever(search_type = 'mmr',
                                    search_kwargs = {'k':5, 'fetch_k': 20, 'lambda_mult': 0.5}
                                    )

  # BM25 retriever
  tokenized_docs = [doc.page_content.split() for doc in all_chunks]
  bm25_db = BM25Okapi(tokenized_docs)
  bm25_retriever = BM25Retriever.from_documents(all_chunks)
  bm25_retriever.k = 5

  return embeddings, faiss_db, faiss_retriever, bm25_retriever

In [ ]:
faiss_embeddings, faiss_db, faiss_retriever, bm25_okapi, bm25_retriever = build_retriever(all_chunks)

In [ ]:
# faiss_retriever 내부의 vectorstore 가져오기
# faiss_db.save_local("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/faiss_db")

# faiss_db = FAISS.load_local(
#     "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/faiss_db",
#     embeddings,
#     allow_dangerous_deserialization=True

# # bm25
# with open("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/bm25_vectordb/bm25_okapi.pkl", "wb") as f:
#     pickle.dump(bm25_okapi, f)

# with open("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/bm25_vectordb/bm25_okapi.pkl", "rb") as f:
#     bm25_db = pickle.load(f)

# Inference

## Define Untils

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9]?\s", line)) for line in lines)
    return option_count >= 2

In [ ]:
# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [ ]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9]?)", text)
        # match = re.search(r"([1-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text

In [ ]:
# 하이브리드 검색기
def hybrid_search(question: str, top_k: int, bm25_weight: int, faiss_weight: int):
  # bm25 점수
  tokenized_question = question.split()
  bm25_scores = bm25_okapi.get_scores(tokenized_question)
  bm25_scores_norm = bm25_scores / (np.max(bm25_scores) + 1e-8)

  # faiss 점수
  ques_embedding = faiss_embeddings.embed_query(question)
  D, I = faiss_vectordb.index.search(np.array([ques_embedding]), len(all_chunks))
  faiss_scores = np.zeros(len(all_chunks))
  faiss_scores[I[0]] = (np.max(D[0]) - D[0]) / (np.max(D[0]) - np.min(D[0]) + 1e-8)

  # 가중합
  combined_scores = bm25_weight * bm25_scores_norm + faiss_weight * faiss_scores

  # Top-K 문서 선택
  top_indices = np.argsort(combined_scores)[::-1][:top_k]
  top_docs = [all_chunks[i] for i in top_indices]

  return top_docs, combined_scores

In [ ]:
# 프롬프트 생성기
def make_prompt_auto(text: str, top_docs: str) -> str:
    """RAG 컨텍스트를 포함해 객관식/주관식 프롬프트를 자동 구성"""
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
            "당신은 금융보안 전문가입니다.\n"
            "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요. 다른 단어/설명 금지.\n\n"
            "예: 1 / 2/ 3/ 4/ 5\n\n"
            f"참고문서: {top_docs}\n\n"
            f"질문: {question}\n"
            "선택지:\n"
            f"{'\n'.join(options)}\n\n"
            "답변:"
        )
    else:
        prompt = (
            "당신은 금융보안 전문가입니다.\n"
            "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
            "단, 참고 문서를 바탕으로 답을 구성하되 검색된 내용을 그대로 복사하지 말고 반드시 **재구성, 요약, 재작성**해서 답변해야 합니다.\n\n"
            f"참고문서: {top_docs}\n\n"
            f"질문: {text}\n\n"
            "답변:"
        )
    return prompt

## Finetunning Model Load

In [ ]:
model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="bfloat16",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
adapter_path = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/finetunning_model8/checkpoint-1104"

# 추론만 고려
fine_model = PeftModelForCausalLM.from_pretrained(model, adapter_path)
fine_model = fine_model.merge_and_unload().to("cuda")

In [ ]:
# 추가 파인튜닝
# fine_model = AutoPeftModelForCausalLM.from_pretrained(
#     adapter_path,
#     torch_dtype="auto",
#     device_map="auto"
# )

# fine_model = PeftModelForCausalLM.from_pretrained(model, adapter_path)
# fine_model = fine_model.merge_and_unload()

# # 여기서 다시 로드 (Hugging Face 방식)
# fine_model.save_pretrained("merged_model")
# tokenizer.save_pretrained("merged_model")

# from transformers import AutoModelForCausalLM, AutoTokenizer
# fine_model = AutoModelForCausalLM.from_pretrained("merged_model").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("merged_model")

# pipe = pipeline("text-generation", model=fine_model, tokenizer=tokenizer)


# pipe = pipeline("text-generation",
#                 model=fine_model,
#                 tokenizer=tokenizer)

## Test data Load


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/test.csv')

## Inference

In [ ]:
def inference(question, fine_model, tokenizer, faiss_vectordb, bm25_okapi,
              top_k: int, bm25_weight: int, faiss_weight: int):

  top_docs = hybrid_search(question, top_k, bm25_weight, faiss_weight)
  prompt = make_prompt_auto(question, top_docs)
  inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

  # 객관식
  if is_multiple_choice(question):
    output_ids = fine_model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
    )
  else:
    output_ids = fine_model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.9
      )

  output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  pred_answer = extract_answer_only(output_text, original_question=q)

  return pred_answer

In [ ]:
torch.cuda.empty_cache()

preds = []

for q in tqdm.tqdm(test['Question'], desc='Inference'):
  answer = inference(q, fine_model, tokenizer, faiss_vectordb, bm25_okapi,
              top_k=7, bm25_weight=0.1, faiss_weight=0.9)
  preds.append(answer)

Inference: 100%|██████████| 515/515 [54:08<00:00,  6.31s/it]


# Submission

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/submission_final.csv', index=False, encoding='utf-8-sig')
sample_submission.to_csv('submission_final.csv', index=False, encoding='utf-8-sig')

files.download('submission_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
preds